In [3]:
import os
import re
from time import sleep
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

def start_search(driver, cafe_info):
    url = "https://www.google.com/maps/"
    # driverPath = "D:\workspace\pythonProject2\chromedriver.exe"
    # driver = webdriver.Chrome(driverPath)
    driver.get(url)
    # result = requests.get(url=url) # 응답

    # search = driver.find_element_by_css_selector("input#searchboxinput.tactile-searchbox-input")
    search = driver.find_element_by_css_selector("#searchboxinput")
    time.sleep(1)

    search.clear()
    search.send_keys(cafe_info.get("name"))
    search.send_keys(Keys.ENTER)
    # driver.implicitly_wait(3)

    return get_store_review_data(driver, cafe_info)


def get_store_review_data(driver, cafe_info):
    while True:
        try:
            time.sleep(5)
            # 더보기 = driver.find_element_by_css_selector('#pane > div > div.widget-pane-content.scrollable-y > div > div > div:nth-child(44) > div > div > button')
            더보기 = driver.find_element_by_css_selector("button[aria-label*='리뷰 더보기']")
            더보기.send_keys(Keys.ENTER)
            # 더보기 = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[41]/div/div/button')
            # time.sleep(1)
            # 더보기.click()
        except Exception as e:
            print(e)
            break

    # last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(100):
        # 스크롤 : 마지막까지
        # reviews = driver.find_elements_by_css_selector('span.section-review-text')
        # for review in reviews:
        #     print(review.text)
        # print(
        #     "========================================================================================================")

        try:
            scroll = driver.find_element_by_css_selector(
            'div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
            time.sleep(1)
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll)
            location1 = scroll.location_once_scrolled_into_view
            print(i, location1)
        except Exception as e:
            print(e)
            break

        time.sleep(1)

    # 스크롤 끝났으니 수집
    reviews = driver.find_elements_by_css_selector('span.section-review-text')
    stars = driver.find_elements_by_css_selector('span.section-review-stars')
    result =[[cafe_info.get("gu"), cafe_info.get("cafe_id"), re.search(r'[0-9]', star.get_attribute("aria-label")).group(), review.text.replace("\n", " ")] for
              star, review in zip(stars, reviews)]
    return result

    # return (stars.get_attribute("aria-label"), reviews.text)
    # print(stars.get_attribute("aria-label"), reviews.text)

def main():

    driverPath = "C:/Users/default.DESKTOP-FVT3076/Desktop/crawl/chromedriver.exe"
    driver = webdriver.Chrome(driverPath)
    df = pd.read_csv('./2021-09-23_리뷰_687행.csv', sep=',', encoding='CP949')
    gu = df['area']
    cafe_id = df['id']
    cafe_name = df['naver_keyword']
    search_name = []
    cafe_len = 10
    for i in range(cafe_len):
        name = cafe_name[i]
        search_name.append({
            "name":name,
            "gu": gu[i],
            "cafe_id": cafe_id[i]
        })

    for i in range(cafe_len):

        result = start_search(driver, search_name[i])

        # 구, ID, stars, 리뷰
        # result = get_store_review_data(driver)
        data = pd.DataFrame(result)
        # data.columns = ['stars', 'reviews']
        # data.columns = ['stars', 'reviews']
        # data = pd.concat([search_name[i].get("gu") + search_name[i].get("cafe_id"), data], axis=1)
        # 누적
        file = 'cafe_reviews.csv'
        if os.path.isfile(file):
            os.remove(file)
        data.to_csv(file, encoding='utf-8')
#         if not os.path.exists("cafe_reviews_GA_GC.csv"):
#             data.to_csv('cafe_reviews.csv', index=False, sep='|', mode="w", encoding='utf-8')
#         else:
#             data.to_csv('cafe_reviews.csv', index=False, sep='|', mode="a", header=False, encoding='utf-8')



"""
1. 카페 정보 불러오기
2. 한라인씩 읽으면 -> 해당 카페의 식별정보(구, ID)
3. 수집 -> [구, ID, stars, review]
"""


if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [11]:
from selenium import webdriver
import time
browser = webdriver.Chrome("C:/Users/default.DESKTOP-FVT3076/Desktop/crawl/chromedriver.exe")
browser.maximize_window()

url = "https://www.google.com/maps/place/24%EC%8B%9C+%EC%A0%84%EC%A3%BC%EB%AA%85%EA%B0%80%EC%BD%A9%EB%82%98%EB%AC%BC%EA%B5%AD%EB%B0%A5+%EC%96%91%EC%82%B0%EC%A0%90/@35.3389164,129.0265622,17z/data=!3m1!4b1!4m5!3m4!1s0x3568978268621293:0x627b62e7d86f9436!8m2!3d35.338912!4d129.0287509"
browser.get(url)
# 페이지 로딩 대기
time.sleep(2)
# prev_height = browser.execute_script("return document.body.scrollHeight")

# 웹페이지 맨 아래까지 무한 스크롤
# while True:
#     # 스크롤을 화면 가장 아래로 내린다
#     browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    
#     # 페이지 로딩 대기
#     time.sleep(2)

#     # 현재 문서 높이를 가져와서 저장
#     curr_height = browser.execute_script("return document.body.scrollHeight")

#     if(curr_height == prev_height):
#         break
#     else:
#         prev_height = browser.execute_script("return document.body.scrollHeight")
# driver.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div[1]/div/button').click()
import time
# time.sleep(3)

# total_number_of_reviews = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]').text.split(" ")[0]
# total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)
# #Find scroll layout
# scrollable_div = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]')
# #Scroll as many times as necessary to load all reviews
# for i in range(0,(round(total_number_of_reviews/10 - 1))):
#         driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
#                 scrollable_div)
#         time.sleep(1)
        
response = BeautifulSoup(browser.page_source, 'html.parser')
reviews = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
def get_review_summary(result_set):
    rev_dict = {'Review Rate': [],
        'Review Time': [],
        'Review Text' : []}
    for result in result_set:
        review_rate = result.find('span', class_='ODSEW-ShBeI-H1e3jb')["aria-label"]
        review_time = result.find('span',class_='ODSEW-ShBeI-RgZmSc-date').text
        review_text = result.find('span',class_='ODSEW-ShBeI-text').text
        rev_dict['Review Rate'].append(review_rate)
        rev_dict['Review Time'].append(review_time)
        rev_dict['Review Text'].append(review_text)
    import pandas as pd    
    return(pd.DataFrame(rev_dict))
res = get_review_summary(reviews)
print(res)
# review_text_crawl_list = browser.find_elements_by_class_name("ODSEW-ShBeI-text")
        
# # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
# review_text_list = []
# for review_crawl_data in review_text_crawl_list:
#         review_text_list.append(review_crawl_data.find_element_by_tag_name('span').text)
# review_text_list

  Review Rate Review Time                                        Review Text
0      별표 5개         3주 전                       3년째야근근무하고.자주갑니다.김치콩나물국밥에.파전굿
1      별표 3개         1년 전  가격대비 괜찮은 곳입니다\n일 때문에 전주에 자주 내려가는데 전주의 콩나물 국밥은 ...
2      별표 4개         1달 전  첨엔 국물이 맑더니 계속 지저분한 빨강으로나옴\n전에 물어보니 월래그렇다는데\n저의...


In [8]:
x = res['Review Rate'].tolist()

' 별표 5개 '